In [7]:
import pandas as pd
import pandapower.networks as nw
from pandapower.cluster.cluster_plotly import cluster_plotly

In [8]:
net = nw.mv_oberrhein("generation")

numb_cluster = 5
cluster_bus = pd.read_excel(f"buses_clustering_{numb_cluster}.xlsx", engine = 'openpyxl', index_col = 0)
cluster_bus = cluster_bus.drop(["bus*"], axis = 1)

cluster_plotly(net, cluster_bus)

KeyError: 'Cluster'